In [1]:
import polars as pl
import os
import numpy as np
import matplotlib.pyplot as plt
from piepy.psychophysics.wheel.detection.wheelDetectionSession import WheelDetectionSession
from piepy.psychophysics.wheel.wheelTrace import WheelTrace
from piepy.core.data_functions import make_subsets

### 1. Analyse the session

In [2]:
exp_name = "230413_KC143_detect_opto120_dorsal__no_cam_KC"
# exp_name = "230106_KC144_detect__no_cam_KC"
w = WheelDetectionSession(exp_name,
                          load_flag=True,
                          skip_google=True)
data = w.runs[0].data.data
paths = w.runs[0].paths

[25-01-30 13:15:07] - Found saved data: /Users/kaan/data/analysis/230413_KC143_detect_opto120_dorsal__no_cam_KC/runData.parquet
[25-01-30 13:15:07] - Loading from ['/Users/kaan/data/analysis/230413_KC143_detect_opto120_dorsal__no_cam_KC']
[25-01-30 13:15:07] - Loaded session data from /Users/kaan/data/analysis/230413_KC143_detect_opto120_dorsal__no_cam_KC/runData.parquet
[25-01-30 13:15:07] - Done! t=0.02 s


/Users/kaan/code/piepy/piepy/psychophysics/wheel/detection/wheelDetectionSession.py:138: OptimizeWarning: Covariance of the parameters could not be estimated
  popt, pcov = curve_fit(


In [ ]:
trial = data[0]

reset_time_point = trial[0,"t_trialstart"]
t = trial[0,"wheel_t"].to_numpy()
pos = trial[0,"wheel_pos"].to_numpy()
trace = WheelTrace()

t,pos = trace.fix_trace_timing(t,pos)
_, _, t_interp, tick_interp = trace.reset_and_interpolate(
                t, pos, reset_time_point, 5
            )

pos_interp = trace.cm_to_rad(trace.ticks_to_cm(tick_interp))

mov_dict = trace.get_movements(
                t_interp,
                pos_interp,
                freq=5,
                pos_thresh=0.00015,  # rads, 0.02 for ticks
                t_thresh=,
            )



In [3]:
from piepy.plotters.psychophysics.progress_plot import *
from piepy.plotters.psychophysics.detection.trial_plotter import plot_trial
from piepy.plotters.psychophysics.detection.cumulative_outcome_plot import plot_cumulative_outcome
from piepy.plotters.psychophysics.detection.psychometric_plot import plot_psychometric
from piepy.plotters.psychophysics.detection.wheel_profile_plot import *
from piepy.plotters.psychophysics.detection.wheel_slope_offset_plot import *
from piepy.plotters.psychophysics.detection.reaction_time_cloud_plot import plot_reaction_time_cloud
from piepy.plotters.psychophysics.detection.reaction_time_distribution_plot import plot_reaction_time_distribution

In [ ]:
import pprint
row = w.runs[0].data.data[245]
pprint.pp(row.to_dict(as_series=False))
print(row[0,"reward"])
plot_trial(row,
           t_thresh=1,
           pos_thresh=0.0005,
           min_dur=20,
           min_gap=30)

In [ ]:
reset_time = 4558891
t = np.array([4559345, 4559481, 4559531, 4559557, 4559579, 4559600, 4559620,
       4559657, 4559724, 4559832, 4559900, 4559965, 4560014, 4560052,
       4560102])

pos = np.array([-423971, -423972, -423972, -423970, -423968, -423966, -423964,
       -423962, -423963, -423965, -423967, -423969, -423971, -423973,
       -423975])
print(len(t),len(pos))

In [ ]:
from scipy.signal import find_peaks 
# trial = row = w.runs[0].data.data[194]

# reset_time_point = trial[0,"t_trialstart"]
# t = trial[0,"wheel_t"].to_numpy()
# pos = trial[0,"wheel_pos"].to_numpy()
trace = WheelTrace()

t,pos = trace.fix_trace_timing(t,pos)
_, _, t_interp, tick_interp = trace.reset_and_interpolate(
                t, pos, reset_time, 5
            )

pos_interp = trace.cm_to_rad(trace.ticks_to_cm(tick_interp))
speed = np.abs(trace.get_filtered_velocity(pos_interp,interp_freq=5))*1000
# plt.plot(t_interp,speed)
plt.plot(t_interp,pos_interp)
plt.plot(t_interp,speed)
mov_dict = trace.get_movements(
                t_interp,
                pos_interp,
                freq=5,
                pos_thresh=0.0003,  # rads, 0.02 for ticks
                t_thresh=1,
                min_dur=20,
                min_gap=30
            )
for i in range(len(mov_dict["onsets"])):
    _t = mov_dict["onsets"][i]
    plt.scatter(_t[1], pos_interp[int(_t[0])], color="b")
    _e = mov_dict["offsets"][i]
    plt.scatter(_e[1], pos_interp[int(_e[0])], color="r")


mov_dict

In [ ]:

# add_t = np.arange(reset_time,t[0],50)
# add_pos = np.array([pos[0]]*len(add_t))

# t = np.append(add_t,t)
# pos = np.append(add_pos,pos)

t = t - reset_time
plt.plot(t,pos)


In [ ]:
_t = np.arange(reset_p,t[0],50)
_pos = np.array([pos[0]]*len(_t))
print(_t)
print(_pos)

In [62]:
new_t = np.append(_t,t)
new_pos = np.append(_pos,pos)

In [ ]:
new_t = new_t - reset_p
plt.plot(new_t,new_pos)


### Progression of performance

In [ ]:
cm = 1/2.54
f,ax = plot_performance(data,
    plot_in_time=True,
    seperate_by=["stimkey","contrast"],
    rolling_window=5,
    mpl_kwargs={"figsize":(10*cm,12*cm),
                "linewidth":3,
                "marker":"o",
                "markersize":4,
                "markeredgewidth":0.3})
# ax.legend(frameon=False)

# date,animalid,*_ = exp_name.split("_")
# savename = f"{date}_{animalid}_hit_rate.pdf"
# p.figs[0].savefig(os.path.join(paths.save[0],"figures",savename))

### Progression of reaction times

In [ ]:
cm = 1/2.54
reaction_of = "reaction_time"
f,ax = plot_reactiontime(
    data,
    plot_in_time=True,
    reaction_of=reaction_of,
    include_misses=False,
    rolling_window=0,
    seperate_by=["stimkey","contrast"],
    mpl_kwargs={"figsize":(10*cm,12*cm),
                "linewidth":3,
                "marker":"o",
                "markersize":4,
                "markeredgewidth":0.3})

### Cumulative Outcomes of the trials

In [ ]:

f,ax = plot_cumulative_outcome(data,reaction_of="response_time",
                               mpl_kwargs={"linewidth":3})

date,animalid,*_ = exp_name.split("_")
savename = f"{date}_{animalid}_resp_distribution.pdf"
f.savefig(os.path.join(paths.save[0],"figures",savename))


### Psychometric curve

In [ ]:
plot_psychometric(data,
                  outcomes=["hit","miss"],
                  style="print")

### Reaction time of trials as dot cloud

In [ ]:
plot_reaction_time_cloud(data,
                         reaction_of="reaction_times",
                         bin_width=10,
                         cloud_width=0.3,
                         mpl_kwargs={"figsize":(12,8),
                                     "s":200,
                                     "linewidths":0.5,
                                     "edgecolors":"w"})

### Distribution of reaction times wrt to stimulus time

In [ ]:
plot_reaction_time_distribution(data,
                                reaction_of="reaction_time",
                                mpl_kwargs={"linewidth":1})

### Wheel Traces

In [ ]:
data_contra = data.filter(pl.col("stim_side")!="ipsi")
plot_all_wheel_profiles(data_contra,
                        plot_speed=False,
                        time_reset="t_vstimstart",
                        mpl_kwargs={"linewidth":1})

### Comparing wheel onsets and peak speeds

In [ ]:
cm = 1/2.54
data_contra = data.filter(pl.col("stim_side")!="ipsi")
f = plot_all_wheel_slope_and_offsets(data_contra,
                                     time_reset="t_vstimstart",
                                     style="print",
                                     mpl_kwargs={"figsize":(15*cm,15*cm),
                                                 "edgecolor":'w',
                                                 "linewidths":0.5})
# date,animalid,*_ = exp_name.split("_")
# savename = f"{date}_{animalid}_slope_vs_offset.pdf"
# f.savefig(os.path.join(paths.save[0],"figures",savename))